In [ ]:
!pip install tiktoken chromadb

In [1]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

raw_docs = [WebBaseLoader(url).load() for url in urls]
docs = [doc for raw_doc in raw_docs for doc in raw_doc]

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap=0)
doc_chunks = splitter.split_documents(docs)
len(doc_chunks)

239

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

vec_db = Chroma.from_documents(
    documents=doc_chunks,
    collection_name="rag",
    embedding=OllamaEmbeddings(model="qwen2.5-coder:14b")
)
retriever = vec_db.as_retriever()

In [4]:
question = "What security issue could we face while using the LLM ?"

In [5]:
# TEST
relevant_docs = retriever.invoke(question)
for relevant_doc in relevant_docs:
    print("==> ", relevant_doc.metadata['source'], relevant_doc.page_content[:80])

==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ Fig. 13. UI for humans to do tool-assisted adversarial attack on a classifier. H
==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ One simple and intuitive way to defend the model against adversarial attacks is 
==>  https://lilianweng.github.io/posts/2023-06-23-agent/ }
]
Challenges#
After going through key ideas and demos of building LLM-centered
==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ In the white-box setting, we have full access to the model parameters and archit


In [6]:
from pydantic import BaseModel, Field

class CheckDocs(BaseModel):
    is_relevant: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

In [7]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen2.5-coder:14b", temperature=0)
llm_structured = llm.with_structured_output(CheckDocs)

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system_instruction = """Return only a 'yes' or 'no' depends on whether the document is relevant to the question"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_instruction),
        ("human", "Document: \n {doc} \n\nQuestion: {question}"),
    ]
)
retriv_eval_chain = prompt | llm_structured

In [9]:
# TEST
for relevant_doc in relevant_docs:
    content = relevant_doc.page_content
    print("is_relevant: ", retriv_eval_chain.invoke({"doc": content, "question": question}), "==> ", relevant_doc.metadata['source'], relevant_doc.page_content[:80])

is_relevant:  is_relevant='yes' ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ Fig. 13. UI for humans to do tool-assisted adversarial attack on a classifier. H
is_relevant:  is_relevant='yes' ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ One simple and intuitive way to defend the model against adversarial attacks is 
is_relevant:  is_relevant='no' ==>  https://lilianweng.github.io/posts/2023-06-23-agent/ }
]
Challenges#
After going through key ideas and demos of building LLM-centered
is_relevant:  is_relevant='yes' ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ In the white-box setting, we have full access to the model parameters and archit


In [10]:
from langchain_core.output_parsers import StrOutputParser

gen_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "You are an assistant for question-answering tasks. If the context is not provided try to figure out the answer yourself, otherwise use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:"
),
    ]
)
gen_resp_chain = gen_prompt | llm | StrOutputParser()

In [11]:
# TEST
def concat_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

resp = gen_resp_chain.invoke({"context": concat_docs(relevant_docs), "question": question})
print(resp)

While using Large Language Models (LLMs), one significant security issue is adversarial attacks, where malicious inputs are crafted to manipulate model outputs. Defending against these attacks by instructing models to avoid harmful content can reduce attack success rates but may also lead to unintended consequences, such as decreased creativity and incorrect interpretation of instructions.


In [12]:
reform_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a question re-writer that converts an input question to a better version that is optimized for web search. Look at the input and try to reason about the underlying semantic meaning."""),
        ("human", "This is the original question: {question} \n Try to rewrite a better one"),
    ]
)

reform_question_chain = reform_prompt | llm | StrOutputParser()

In [13]:
# TEST
question, reform_question_chain.invoke({"question": question})

('What security issue could we face while using the LLM ?',
 'What are the potential security risks associated with using large language models (LLMs)?')

In [14]:
from typing import List
from typing_extensions import TypedDict

class State(TypedDict):
    docs: List[str]
    question: str
    retry: int
    gen_without_context: str
    resp: str

In [15]:
def init_state(state):
    print("*** INIT STATE ***")
    
    return {"retry": 3}

def retrv_docs(state):
    print("*** RETRIEVE DOCS ***")
    print("  current state: ", state)
    
    docs = retriever.invoke(state["question"])
    return {"docs": docs}

def eval_docs(state):
    print("*** CHECK DOCS RELEVANCE ***")
    related_docs = []
    gen_without_context = "FALSE"

    for doc in state["docs"]:
        eval_rslt = retriv_eval_chain.invoke({"doc": doc.page_content, "question": state["question"]})
        print("  ==> ", doc.metadata['source'], doc.page_content[:80], " is relevant: ", eval_rslt.is_relevant)
        if eval_rslt.is_relevant == "yes":
            related_docs.append(doc)
    if len(related_docs) == 0:
        print("  !!! NO RELATED DOCS FOUND !!!")
        gen_without_context = "TRUE"

    return {"docs": related_docs, "gen_without_context": gen_without_context}

def reform_question(state):
    print("*** REFORM QUESTION ***")
    print("  ORIGINAL QUESTION: ", state["question"])

    reformed_question = reform_question_chain.invoke({"question": state["question"]})
    print("  REFORMED QUESTION: ", reformed_question)
    return {"question": reformed_question, "retry": state["retry"] - 1}

def gen_resp(state):
    print("*** GENERATE RESPONSE ***")

    resp = gen_resp_chain.invoke({"context": state["docs"], "question": state["question"]})
    return {"resp": resp}

def retry_or_gen(state):
    print("*** MAKING DECISION ***")
    if state["gen_without_context"] == "FALSE":
        print("  WILL GENERATE RESPONSE")
        return "do_gen_resp"

    if state["retry"] == 0:
        print("  UNABLE TO RETRIEVE DOC, WILL GENERATE RESPONSE")
        return "do_gen_resp"

    print("  TRY TO REFORM THE QUESTION AND RETRIEVE AGAIN")
    return "do_reform_question"

In [16]:
from langgraph.graph import START, END, StateGraph

graph = StateGraph(State)
graph.add_node("init_state", init_state)
graph.add_node("retrv_docs", retrv_docs)
graph.add_node("eval_docs", eval_docs)
graph.add_node("reform_question", reform_question)
graph.add_node("gen_resp", gen_resp)

graph.add_edge(START, "init_state")
graph.add_edge("init_state", "retrv_docs")
graph.add_edge("retrv_docs", "eval_docs")
graph.add_conditional_edges(
    "eval_docs",
    retry_or_gen,
    {
        "do_gen_resp": "gen_resp",
        "do_reform_question": "reform_question",
    }
)
graph.add_edge("reform_question", "retrv_docs")
graph.add_edge("gen_resp", END)

workflow = graph.compile()

In [17]:
# TEST
user_input = {"question": "What security issue could we face while using the LLM ?"}
for stdout in workflow.stream(user_input):
    for key, value in stdout.items():
        print("node: ", key)

print(value["resp"])

*** INIT STATE ***
node:  init_state
*** RETRIEVE DOCS ***
  current state:  {'question': 'What security issue could we face while using the LLM ?', 'retry': 3}
node:  retrv_docs
*** CHECK DOCS RELEVANCE ***
  ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ Fig. 13. UI for humans to do tool-assisted adversarial attack on a classifier. H  is relevant:  yes
  ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ One simple and intuitive way to defend the model against adversarial attacks is   is relevant:  yes
  ==>  https://lilianweng.github.io/posts/2023-06-23-agent/ }
]
Challenges#
After going through key ideas and demos of building LLM-centered  is relevant:  no
  ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ In the white-box setting, we have full access to the model parameters and archit  is relevant:  yes
*** MAKING DECISION ***
  WILL GENERATE RESPONSE
node:  eval_docs
*** GENERATE RESPONSE ***
node:  gen_resp
Adversarial attack

In [18]:
# TEST
user_input = {"question": "What's kubernetes ?"}
for stdout in workflow.stream(user_input):
    for key, value in stdout.items():
        print("node: ", key)

print(value["resp"])

*** INIT STATE ***
node:  init_state
*** RETRIEVE DOCS ***
  current state:  {'question': "What's kubernetes ?", 'retry': 3}
node:  retrv_docs
*** CHECK DOCS RELEVANCE ***
  ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ Disclaimer: Not trying to be comprehensive here. Need a separate blog post to go  is relevant:  no
  ==>  https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/ There are a couple strategies for how to update in-context examplars in FLIRT:

  is relevant:  no
  ==>  https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/ Prompt Engineering, also known as In-Context Prompting, refers to methods for ho  is relevant:  no
  ==>  https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/ Product-of-Experts (PoE), combines all probabilities used above in addition to $  is relevant:  no
  !!! NO RELATED DOCS FOUND !!!
*** MAKING DECISION ***
  TRY TO REFORM THE QUESTION AND RETRIEVE AGAIN
node:  eval_docs
*** REFORM QUESTION ***
  O